In [1]:
# Imports
import re
import cv2
import os
from imutils import paths
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import numpy as np
import shutil
import random
from urllib import request

In [2]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
# Check TensorFlow version
print("TensorFlow version:", tf.__version__)

# List available physical devices
physical_devices = tf.config.list_physical_devices()
print("Available physical devices:", physical_devices)

# List GPUs
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPUs available:")
    for gpu in gpus:
        print(gpu)
else:
    print("No GPUs available.")

TensorFlow version: 2.9.0
Available physical devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPUs available:
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [4]:
os.getcwd()

'/Users/lfap/Documents/BSE/T3/DeepLearning/Part2/DL_Poster'

In [4]:
# Step 1: Get the current working directory
current_directory = os.getcwd()
print("Current Directory:", current_directory)

# Step 2: Navigate up one level to the parent directory
parent_directory = os.path.dirname(current_directory)
print("Parent Directory:", parent_directory)

# Step 3: Enter a specified folder from the parent directory
specified_folder = "00 Data/train"  # Replace with the name of your folder
target_directory = os.path.join(parent_directory, specified_folder)
print("Target Directory:", target_directory)

# # Change to the target directory
# os.chdir(target_directory)
# print("Changed to Target Directory:", os.getcwd())

Current Directory: /Users/lfap/Documents/BSE/T3/DeepLearning/Part2/DL_Poster
Parent Directory: /Users/lfap/Documents/BSE/T3/DeepLearning/Part2
Target Directory: /Users/lfap/Documents/BSE/T3/DeepLearning/Part2/00 Data/train


In [5]:
# Load the file paths for all available images from the competition
all_image_paths = list(paths.list_images(target_directory)) 
print(f"Total images: {int(len(all_image_paths))}")

# Shuffle the list of images
random.seed(33)  
random.shuffle(all_image_paths)

# Calculate the split indices
train_size = int(0.7 * len(all_image_paths))
val_test_size = int(0.15 * len(all_image_paths))

# Split the dataset
train_paths = all_image_paths[:train_size]
val_paths = all_image_paths[train_size:train_size + val_test_size]
test_paths = all_image_paths[train_size + val_test_size:]

labels = pd.read_csv('train_labels.csv')

# Print the number of images in each set
print(f"Training set size: {len(train_paths)}")
print(f"Validation set size: {len(val_paths)}")
print(f"Test set size: {len(test_paths)}")

Total images: 11541
Training set size: 8078
Validation set size: 1731
Test set size: 1732


In [6]:
# Extract the ID from each image file path
def extract_id(file_path):
    base_name = os.path.basename(file_path)
    file_id, _ = os.path.splitext(base_name)
    return file_id

# Create a DataFrame from the image paths
image_data = {
    'file_path': all_image_paths,
    'id': [extract_id(path) for path in all_image_paths]
}
image_df = pd.DataFrame(image_data)

# Merge the image paths with the labels
merged_df = pd.merge(image_df, labels, on='id', how='inner')


### We only had labels for one dataset, therefore we will create subsets from the original 'train' dataset and test on the remaining 30% from the split (both validation and test)

In [7]:
# Split merged_df into training, validation, and test DataFrames
train_df = merged_df[merged_df['file_path'].isin(train_paths)]
val_df = merged_df[merged_df['file_path'].isin(val_paths)]
test_df = merged_df[merged_df['file_path'].isin(test_paths)]

In [8]:
# Define the data generator for training with augmentations
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True
)

# Define the ImageDataGenerator for validation and test (without augmentations)
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Specify the columns for the generator
image_column = 'file_path'
label_columns = labels.columns[1:]  # all columns except 'id' are labels

# Create training generator
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col=image_column,
    y_col=label_columns.tolist(),
    target_size=(224, 224),  # Adjust based on your requirements
    batch_size=32,
    class_mode='raw',  # Use 'raw' for multi-label classification
    shuffle=True
)

# Create validation generator
validation_generator = val_test_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col=image_column,
    y_col=label_columns.tolist(),
    target_size=(224, 224),  # Adjust based on your requirements
    batch_size=32,
    class_mode='raw',  # Use 'raw' for multi-label classification
    shuffle=True
)

# Create test generator
test_generator = val_test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col=image_column,
    y_col=label_columns.tolist(),
    target_size=(224, 224),  # Adjust based on your requirements
    batch_size=32,
    class_mode='raw',  # Use 'raw' for multi-label classification
    shuffle=False
)

Found 8078 validated image filenames.
Found 1731 validated image filenames.
Found 1732 validated image filenames.


In [9]:
# Fit model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(label_columns), activation='softmax')  # Multi-label output
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

2024-06-21 13:26:09.933643: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-06-21 13:26:09.934370: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M2

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

Epoch 1/10


2024-06-21 13:26:10.334593: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-06-21 13:26:10.636256: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


253/253 [==============================] - ETA: 0s - loss: 0.5153 - accuracy: 0.4007

2024-06-21 13:26:39.916927: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


253/253 [==============================] - 34s 131ms/step - loss: 0.5153 - accuracy: 0.4007 - val_loss: 0.2633 - val_accuracy: 0.5500
Epoch 2/10
253/253 [==============================] - 32s 127ms/step - loss: 0.2200 - accuracy: 0.6375 - val_loss: 0.2278 - val_accuracy: 0.6101
Epoch 3/10
253/253 [==============================] - 32s 127ms/step - loss: 0.1711 - accuracy: 0.7353 - val_loss: 0.1944 - val_accuracy: 0.6869
Epoch 4/10
253/253 [==============================] - 32s 127ms/step - loss: 0.1438 - accuracy: 0.7868 - val_loss: 0.1713 - val_accuracy: 0.7562
Epoch 5/10
253/253 [==============================] - 32s 127ms/step - loss: 0.1204 - accuracy: 0.8295 - val_loss: 0.1890 - val_accuracy: 0.7516
Epoch 6/10
253/253 [==============================] - 32s 127ms/step - loss: 0.1082 - accuracy: 0.8522 - val_loss: 0.1873 - val_accuracy: 0.7655
Epoch 7/10
253/253 [==============================] - 32s 128ms/step - loss: 0.0961 - accuracy: 0.8713 - val_loss: 0.1678 - val_accuracy: 0.7

In [26]:
# Fit model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(label_columns), activation='sigmoid') 
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')


Found 11541 validated image filenames.
Found 2473 validated image filenames.
Found 2474 validated image filenames.
Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2024-06-13 14:59:33.689549: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-06-13 14:59:33.691020: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/10


2024-06-13 14:59:34.584839: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-06-13 14:59:34.788231: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x32x1x8xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be

122/361 [=========>....................] - ETA: 38s - loss: 1.0297 - accuracy: 0.3266

...

2024-06-13 15:00:35.543273: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x32x1x8xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x32x1x8xi1>'


361/361 [==============================] - 65s 175ms/step - loss: 0.5180 - accuracy: 0.4742 - val_loss: 0.2253 - val_accuracy: 0.6235
Epoch 2/10


loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x9x1x8xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x9x1x8xi1>'


361/361 [==============================] - 62s 172ms/step - loss: 0.1949 - accuracy: 0.6877 - val_loss: 0.1863 - val_accuracy: 0.7145
Epoch 3/10
361/361 [==============================] - 62s 172ms/step - loss: 0.1542 - accuracy: 0.7712 - val_loss: 0.1659 - val_accuracy: 0.7416
Epoch 4/10
361/361 [==============================] - 63s 175ms/step - loss: 0.1275 - accuracy: 0.8199 - val_loss: 0.1524 - val_accuracy: 0.7901
Epoch 5/10
361/361 [==============================] - 62s 170ms/step - loss: 0.1092 - accuracy: 0.8510 - val_loss: 0.1415 - val_accuracy: 0.8055
Epoch 6/10
361/361 [==============================] - 61s 170ms/step - loss: 0.0933 - accuracy: 0.8710 - val_loss: 0.1445 - val_accuracy: 0.8051
Epoch 7/10
361/361 [==============================] - 62s 171ms/step - loss: 0.0852 - accuracy: 0.8864 - val_loss: 0.1432 - val_accuracy: 0.8285
Epoch 8/10
361/361 [==============================] - 61s 170ms/step - loss: 0.0768 - accuracy: 0.8991 - val_loss: 0.1342 - val_accuracy: 0.8

loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x10x1x8xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x10x1x8xi1>'


### ------------------------------------------------------------

In [9]:
base_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model

model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(label_columns), activation='sigmoid')
])

Metal device set to: Apple M2

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2024-06-23 11:57:47.845368: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-06-23 11:57:47.846010: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


58889256/58889256 [==============================] - 30s 1us/step


In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)

Epoch 1/10


2024-06-23 11:58:59.580221: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-06-23 11:58:59.832909: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


253/253 [==============================] - ETA: 0s - loss: 0.3488 - accuracy: 0.3484

2024-06-23 12:00:47.039267: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


253/253 [==============================] - 131s 516ms/step - loss: 0.3488 - accuracy: 0.3484 - val_loss: 0.2720 - val_accuracy: 0.4801
Epoch 2/10
253/253 [==============================] - 165s 652ms/step - loss: 0.2945 - accuracy: 0.4466 - val_loss: 0.2477 - val_accuracy: 0.5661
Epoch 3/10
253/253 [==============================] - 184s 727ms/step - loss: 0.2762 - accuracy: 0.4915 - val_loss: 0.2427 - val_accuracy: 0.5841
Epoch 4/10
253/253 [==============================] - 189s 748ms/step - loss: 0.2643 - accuracy: 0.5053 - val_loss: 0.2329 - val_accuracy: 0.6002
Epoch 5/10
253/253 [==============================] - 188s 742ms/step - loss: 0.2536 - accuracy: 0.5409 - val_loss: 0.2200 - val_accuracy: 0.6222
Epoch 6/10
253/253 [==============================] - 187s 739ms/step - loss: 0.2450 - accuracy: 0.5616 - val_loss: 0.2165 - val_accuracy: 0.6256
Epoch 7/10
253/253 [==============================] - 192s 757ms/step - loss: 0.2381 - accuracy: 0.5751 - val_loss: 0.2133 - val_accura

In [12]:
model.save_weights('TransferLearningWeights.h5')

In [10]:
# Import necessary modules
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

# Load the EfficientNetB0 model with pre-trained weights, excluding the top layer
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom top layers for multi-label classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout to prevent overfitting
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout to prevent overfitting
predictions = Dense(len(label_columns), activation='sigmoid')(x)  # Use sigmoid for multi-label classification

# Combine the base model and the custom top layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
tensorboard = TensorBoard(log_dir='./logs')

# Train the model with frozen base layers
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
    callbacks=[checkpoint, tensorboard]
)

# Unfreeze the last 20 layers of the base model
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Continue training the model with unfrozen layers
history_fine_tune = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
    callbacks=[checkpoint, tensorboard]
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

Epoch 1/10


2024-06-21 13:08:33.297000: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-06-21 13:08:34.863332: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


253/253 [==============================] - ETA: 0s - loss: 0.3855 - accuracy: 0.1437

2024-06-21 13:09:13.895468: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


253/253 [==============================] - 50s 185ms/step - loss: 0.3855 - accuracy: 0.1437 - val_loss: 0.3776 - val_accuracy: 0.1542
Epoch 2/10
253/253 [==============================] - 44s 172ms/step - loss: 0.3768 - accuracy: 0.1451 - val_loss: 0.3842 - val_accuracy: 0.1421
Epoch 3/10
253/253 [==============================] - 46s 182ms/step - loss: 0.3756 - accuracy: 0.1469 - val_loss: 0.3854 - val_accuracy: 0.1421
Epoch 4/10
127/253 [==============>...............] - ETA: 22s - loss: 0.3754 - accuracy: 0.1464

KeyboardInterrupt: 